In [3]:
#deep fake detection
import tensorflow as tf
import keras 
from keras.layers import Activation, Dense
from keras.layers import Conv1D,Conv2D
from keras.models import Sequential
from keras import backend as k



successful


In [5]:
#gpu accelaration on online editors
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
#CODING THE GENERATIVE ADVERSARIAL  NETWORK
#oading the functions
#dense--> noise layer of the network
#conv2dtranspose--> it enalbes us to convolve and upsacale the image at the same time
#it is equivalent to upsampling2d followed by conv2d
#LeakyRelu-->avoids gardeint vainsh problem
#BatchNormalisation--> helps to normalise the covultion result
#reshape--> helps to convert 1D array to 3D

